# Graham scan algorithm

Use the cell below for all python code needed to realise the Graham scan algorithm (including any auxiliary data structures and functions you might need). The `grahamscan()` function itself should take as input parameter a list of 2D points (`inputSet`), and return the subset of such points that lie on the convex hull (`outputSet`).

In [ ]:
from math import atan2

def bottom_most(points: list) -> tuple:
    """Function that fetches the bottom most point in the set of points given

    Args:
        points (list): a list of di-tuples representing x,y coordinates
    
    Returns:
        (tuple): the bottom most point in the set of points given
    """
    ref = points[0]
    for point in points:
        if point[1] < ref[1] or (point[1] == ref[1] and point[0] < ref[0]):
            ref = point

    return ref

def calc_polar_angle(p1: tuple ,p2: tuple) -> float:
    """Function that gets the polar angle from two points by using arctan on the y-distance and x-distance
    
    Args:
        p1 (tuple) : a di-tuple (x,y) representing the first coordinate,
        p2 (tuple) : a di-tuple (x,y) representing the second coordinate.

    Returns:
        (float) : the polar angle between p1 and p2 in radians.
    """
    return atan2(p2[1] - p1[1], p2[0] - p1[0])

def distance(p1: tuple, p2: tuple) -> float:
    """Function that calculates the distance between two given points    
    
    Args:
        p1 (tuple) : a di-tuple (x,y) representing the first coordinate,
        p2 (tuple) : a di-tuple (x,y) representing the second coordinate.

    Returns:
        (float) : the distance between p1 and p2.
    """

    return (p2[1] - p1[1]) **2 + (p2[0] - p1[0]) **2

def determinant(p1: tuple, p2: tuple, p3: tuple) -> float:
    """Function that gets the determinant for p1, p2 and p3, to determine whether it is a clockwise or
       anticlockwise turn. 

        Args:
        p1 (tuple) : a di-tuple (x,y) representing the first coordinate,
        p2 (tuple) : a di-tuple (x,y) representing the second coordinate,
        p3 (tuple) : a di-tuple (x,y) representing the third coordinate,

    Returns:
        (float) : the determinant between p1, p2 and p3.

    Note:
        det > 0 -> clockwise turn
        det < 0 -> anticlockwise turn
        det == 0 -> points are collinear
    """

    return (p2[0]-p1[0]) * (p3[1]-p1[1])  - (p2[1]-p1[1]) * (p3[0]-p1[0])


In [ ]:
def merge(angles: dict, points: list, aux: list, ref: tuple, low: int, mid: int, high: int):
    """The merge step of the merge sort.

    Args:
        angles (dict): dictionary that has the polar angle of each point in points,
        points (list): a list of di-tuples representing (x,y) coordinates,
        aux (list): the auxiliary list,
        ref (tuple): the reference tuple that is used to calculate distance in the case where there are,
                     multiple points with the same polar angle,
        low (int): the index at the start of the partition,
        mid (int): the index at the middle of the partition,
        high (int): the index at the end of the partition.
    """
    # copy over into the auxiliary list.
    for i in range(low, high+1):
        aux[i] = points[i]

    first_partition_counter = low
    second_partition_counter = mid+1

    for counter in range(low, high+1):
        if first_partition_counter > mid: # if larger then there are no more elements in the first_partition
            points[counter] = aux[second_partition_counter]
            second_partition_counter = second_partition_counter + 1
            
        elif second_partition_counter > high: # if smaller then there are no more elements in the second_partition
            points[counter] = aux[first_partition_counter]
            first_partition_counter = first_partition_counter + 1

        elif angles[aux[second_partition_counter]] < angles[aux[first_partition_counter]]: # compare the polar angles
            points[counter] = aux[second_partition_counter]
            second_partition_counter = second_partition_counter + 1
        
        elif angles[aux[second_partition_counter]] == angles[aux[first_partition_counter]]: # if angles are equal compare distance.
            if distance(aux[second_partition_counter], ref) < distance(aux[first_partition_counter], ref):
                points[counter] = aux[second_partition_counter]
                second_partition_counter = second_partition_counter + 1

        else:
            points[counter] = aux[first_partition_counter]
            first_partition_counter = first_partition_counter + 1

def mergesort(angles: dict, points: list, aux: list, ref: tuple, low: int, high: int):
    """Recursive merge sort that sorts points based on their polar angle with the reference point.

    Args:
        angles (dict): dictionary that has the polar angle of each point in points,
        points (list): a list of di-tuples representing (x,y) coordinates,
        aux (list): the auxiliary list,
        ref (tuple): the reference tuple that is used to calculate distance in the case where there are,
                     multiple points with the same polar angle,
        low (int): the index at the start of the partition,
        high (int): the index at the end of the partition.
    """
    if (high <= low): # base case
        return
    
    mid = low + ((high - low)//2)

    # splitting the list.
    mergesort(angles, points, aux, ref, low, mid)
    mergesort(angles, points, aux, ref, mid+1, high)
    if angles[points[mid]] < angles[points[mid+1]]: # if already in the correct order then skip merge
        return

    merge(angles, points, aux, ref, low, mid, high) # merge the two partitions.


def grahamscan(inputSet: list) -> list:
    """Function that scans through the list and returns a list of points that are on the convex hull

    Args:
        inputSet (list): a list of di-tuples representing (x,y) coordinates

    Returns:
        (list): a list of di-tuples representing (x,y) coordinates on the convex hull.
    """
    
    ref = bottom_most(inputSet) # the bottom most point is the reference point.

    # placed in a dictionary to avoid recalculation
    angles = dict([(tuple(point), calc_polar_angle(point, ref)) for point in inputSet])

    mergesort(angles,inputSet, [None]*len(inputSet), ref, 0, len(inputSet)-1) # sort based on angle made with reference point
    stack = []
    for point in inputSet:
        while len(stack) > 1 and determinant(stack[-2], stack[-1],point) < 0:
            #removes points on the stack that make an anticlockwise turn
            del stack[-1]
        stack.append(point) #adds points that could potentially be on the convex hull.
    
    return stack

Use the cell below for all python code needed to generate test data points (both random and those representing worst-case scenario).

In [ ]:
import random

#code for random data generation
def generatePoint() -> tuple:
    """Function that generates a random valid point"""
    return (random.randint(0, 32767), random.randint(0, 32767))

def generatePoints(limit: int) -> list:
    """Function that generates a random list of point"""
    return [generatePoint() for point in range(limit)]

#code for worst case data generation
   
def join(points: list, left: list, right: list, low: int, mid: int, high: int):
    """Functions that merges two partitions together for the worst_merge_sort procedure,
    in a way that triggers the worst case for merge sort.

    Args:
        points (list): the original list of points, which the left and right lists are to merge into,
        left (list): the list of points in the left partition,
        right (list): the list of points in the right partition,
        low (int): the index at which the left partition begins at.
        mid (int): the index at which the left partition ends, and the right one begins.
        high (int): the index at which the right partition ends.
    """
    for i in range(mid-low+1):
        points[i] = left[i]
    
    for j in range(high-mid):
        points[i + j + 1] = right[j]


def worst_merge_sort(points: list, low: int, high: int):
    """Recursive procedure that changes the order of the sorted list of points to trigger the 
    worst case for merge sort, which is a list that triggers the maximum number of comparisons.

    Args:
        points (list): a sorted list,
        low (int): the start index of the partition,
        high (int): the end index of the partition.
    """
    if low < high:
        mid = low + ((high - low) // 2)

        left = [points[i*2] for i in range(mid-low+1)] # takes even indices
        right = [points[i * 2 + 1] for i in range(high-mid)] # takes odd indices

        # partition again.
        worst_merge_sort(left, low, mid)
        worst_merge_sort(right, mid+1, high)

        join(points, left, right, low, mid , high)

def worstCase(limit: int) -> list:
    """Function that generates the worst case scenario for the graham scan

    Args:
        limit (int): the number of points that are to be in the list.
    
    Returns: 
        (list) the list of points that is structured to trigger the worst case for graham.
    """
    inputSet = generatePoints(limit) # generate a random set of points.
    ref = bottom_most(inputSet) # get the reference point.

    # create angles dictionary.
    angles = dict([(point, calc_polar_angle(point, ref)) for point in inputSet])

    mergesort(angles, inputSet, [None]*len(inputSet), ref, 0, len(inputSet)-1) # sort inputSet
    worst_merge_sort(inputSet, 0, len(inputSet)-1) # transform it into worst case

    return inputSet # return worst case

Use the cell below for all python code needed to test the `grahamscan()` function on the data generated above.

In [ ]:
import timeit

#test code

def run_tests(limits: list, data_generator, title="", tests=100):
    """Procedure that tests the code, and presents the findings in a prettified manner.

    Args:
        limits (list): a list of limits to test the grahamscan on,
        data_generator (function): a function, that when given a limit from limits, generates a list of points,
        title (str): title is used as a label,
        tests (int): number of times to run a limit. Defaults to 100.
    """
    
    print("="*10, f" STARTING: {title} cases ", "="*10, "\n")
    for limit in limits:
        # take avg. data generation time to subtract from total time, to get time taken for grahamscan.
        data_generation_time = timeit.timeit('data_generator(limit)', number = tests, globals={'data_generator':data_generator, 'limit':limit})

        # get the avg. time taken to grahamscan a generated list (a new list is generated each iteration), includes time for data generation.
        total_time = timeit.timeit('grahamscan(data_generator(limit))', number = tests,  
        globals={'grahamscan':grahamscan,'data_generator':data_generator, 'limit':limit})

        # subtract total_time by the time it takes to generate data, and then take avg.
        grahamscan_time = (total_time - data_generation_time) / tests
        
        string =  f'{title} Case @ {limit}: {grahamscan_time} seconds' 
        print(string)

    print()
    print("="*10, f" FINISHED: {title} cases ", "="*10)


limits = [100, 500, 1000, 5000, 10000, 15000, 20000]
run_tests(limits, generatePoints, "Normal")
run_tests(limits, worstCase, "Worst")


*Optional*: Feel free to use the code below on small datasets (e.g., N = 10) to visually inspect whether the algorithm has been implemented correctly. The fragment below assumes both `inputSet` and `outputSet` to be lists of 2D points, with each point being a list of 2 elements (e.g., `[[x1,y1], [x2,y2], ..., [x_k,y_k]]`)

In [ ]:
import matplotlib.pyplot as plt

# inputSet and outputSet should have been defined above. 
# uncomment the next two lines only if you wish to test the plotting code before coding your algorithm

#inputSet = [[1,1], [2,2] , [3, 3], [4,4], [1,4], [3,1], [1, 5], [2, 4], [3, 5]]
#outputSet = [[1,1], [3,1] , [4, 4], [3,5], [1,5]]

inputSet = generatePoints(200)
outputSet = grahamscan(inputSet)

plt.figure()

#first do a scatter plot of the inputSet
input_xs, input_ys = zip(*inputSet)
plt.scatter(input_xs, input_ys)

#then do a polygon plot of the computed covex hull
outputSet.append(outputSet[0]) #first create a 'closed loop' by adding the first point at the end of the list
output_xs, output_ys = zip(*outputSet)
plt.plot(output_xs, output_ys) 

plt.show() 